## 矩阵分解or 矩阵补充算法（过时&不适合做召回）

原理
1) 计算物品emb 和用户emb
2) 使用dot计算 user_emb, item_emb见的内积
3) 内积结果用来拟合user给item的打分值

缺点：
1) 仅⽤ID embedding，没利⽤物品、⽤户属性
2) 负样本：曝光之后，没有点击、交互。（错误的做法）
3) 做训练的⽅法不好(回归不如分类，内积不如余弦相似度)
4) 对新物品不友好( 新物品没有展现数据，rate数据，更没有训练好的emb)


In [75]:
%cd /playground/sgd_deep_learning/sgd_rec_sys/
import sys 
sys.path.append('./python')

/playground/sgd_deep_learning/sgd_rec_sys


In [76]:
import numpy as np
import random
import torch
from sgd_rec_sys.retrieval import MF, RateInfo

## 准备数据

In [77]:
# TODO: 改写成DataSet 和 DataLoader
def load_data():
    data = [(1, 2, 4.5),
            (1, 3, 2.0),
            (2, 1, 4.0), 
            (2, 3, 3.5),
            (3, 2, 5.0),
            (3, 4, 2.0),
            (4, 2, 3.5), 
            (4, 3, 4.0), 
            (4, 4, 1.0)]
    uids = [x[0] - 1 for x in data]
    iids = [x[1] - 1 for x in data]
    scores = [x[2] for x in data]

    return torch.LongTensor(uids), torch.LongTensor(iids),  torch.Tensor(scores)

user_ids, item_ids, scores = load_data() 
print(user_ids, item_ids, scores)

tensor([0, 0, 1, 1, 2, 2, 3, 3, 3]) tensor([1, 2, 0, 2, 1, 3, 1, 2, 3]) tensor([4.5000, 2.0000, 4.0000, 3.5000, 5.0000, 2.0000, 3.5000, 4.0000, 1.0000])


In [78]:
def train(model, train_x, loss_fn, optimizer):
    user_ids, item_ids, scores = train_x
    y = model.forward(user_ids, item_ids)
    out = loss_fn(y, scores)

    # Backpropagation
    optimizer.zero_grad()
    out.backward()
    optimizer.step()
    return out

In [79]:
mf = MF(userN=4, itemN=4, dim=32)
print(mf)

training_data = load_data()
loss_fn = torch.nn.MSELoss()
# optimizer = torch.optim.SGD(mf.parameters(), lr=1e-2)


best_states = None
best_loss = None

for epoch in range(50):
    optimizer = torch.optim.Adam(mf.parameters(), lr=1e-2, weight_decay=0.1)
    loss = train(model=mf, train_x=training_data, loss_fn=loss_fn, optimizer=optimizer)
    
    if best_loss is not None and loss > best_loss:
        continue
    else:
        best_loss = loss
        best_states = mf.state_dict()
    
    print(epoch, 'loss:', loss)

# load best parameters
mf.load_state_dict(best_states)

MF(
  (user_emb): Embedding(4, 32)
  (item_emb): Embedding(4, 32)
)
0 loss: tensor(18.5207, grad_fn=<MseLossBackward0>)
1 loss: tensor(15.8223, grad_fn=<MseLossBackward0>)
2 loss: tensor(13.3671, grad_fn=<MseLossBackward0>)
3 loss: tensor(11.1465, grad_fn=<MseLossBackward0>)
4 loss: tensor(9.1638, grad_fn=<MseLossBackward0>)
5 loss: tensor(7.4099, grad_fn=<MseLossBackward0>)
6 loss: tensor(5.8869, grad_fn=<MseLossBackward0>)
7 loss: tensor(4.5793, grad_fn=<MseLossBackward0>)
8 loss: tensor(3.4899, grad_fn=<MseLossBackward0>)
9 loss: tensor(2.6018, grad_fn=<MseLossBackward0>)
10 loss: tensor(1.8963, grad_fn=<MseLossBackward0>)
11 loss: tensor(1.3427, grad_fn=<MseLossBackward0>)
12 loss: tensor(0.9196, grad_fn=<MseLossBackward0>)
13 loss: tensor(0.5939, grad_fn=<MseLossBackward0>)
14 loss: tensor(0.3528, grad_fn=<MseLossBackward0>)
15 loss: tensor(0.1799, grad_fn=<MseLossBackward0>)
16 loss: tensor(0.0721, grad_fn=<MseLossBackward0>)
17 loss: tensor(0.0181, grad_fn=<MseLossBackward0>)
18

<All keys matched successfully>

## 验证训练集效果


In [80]:
# 重复运行，查看不同样本预测情况
k = random.randint(0, len(training_data[0])-1)
user_id, item_id, rate_score = [training_data[i][k] for i in range(3)]

pred_y = mf.forward(user_id.unsqueeze(-1), item_id.unsqueeze(-1))
print("user[{}], item[{}], pred_rate[{}], True_rate[{}]".format(user_id, item_id, rate_score, pred_y))

user[0], item[2], pred_rate[2.0], True_rate[tensor([2.0260], grad_fn=<SumBackward1>)]


## emb线上使用流程
* user emb 使用userid 即可查找
* item emb 需要使用Milvus、Faiss、HnswLib等向量数据库，方便快速找到与query(user emb)内积最大的topk物品